In [1]:
img_dir = '/home/fmr/Downloads/scalpel/rescale/images'
label_dir = '/home/fmr/Downloads/scalpel/rescale/jsons'


In [2]:
import cv2
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR,ReduceLROnPlateau

from dataset import CustomDataset
from dataset_aug import CustomDatasetAugmented
from dataset_aug_albu import CustomDatasetAlbu
from loss import compute_loss, compute_loss_smooth_focal
from network import SimpleObjectDetector, SimpleObjectDetectorDropout,SimpleObjectDetectorRed, SimpleObjectDetectorBN, \
SimpleObjectDetectorWithResnet, FPN,SimpleObjectDetectorRedInput

from network_attention import AttentionObjectDetector
import albumentations as A


In [6]:
INPUT_SIZE = (512, 512 )
BATCH_SIZE = 64
NUM_WORKERS = 16

NUM_EPOCHS = 4000
SAVE_EVERY = 40
MAX_OBJECTS =5
num_classes = 5

# Model
#model = SimpleObjectDetector(input_width=224, input_height=224, num_classes=5, max_objects=5)
#model = SimpleObjectDetector()
#model = SimpleObjectDetectorWithResnet()
#model = FPN()
#model = SimpleObjectDetectorRed()
model = SimpleObjectDetectorRedInput(input_size=INPUT_SIZE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [3]:
from torchsummary import summary

# Assuming the model is defined and is named 'model'
input_size = (3, 512, 512)  # for example, adjust if needed
summary(model, input_size)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [8]:
for param in model.fc_class.parameters():
    print(param)

Parameter containing:
tensor([[ 5.4598e-02, -1.2611e-02,  1.5817e-03,  ...,  4.7906e-02,
          1.7106e-02,  2.0466e-02],
        [ 1.1705e-02,  2.3764e-02,  6.2032e-02,  ..., -6.7343e-03,
         -3.4995e-02, -4.9746e-02],
        [ 1.9897e-02,  1.6289e-02,  4.1104e-02,  ..., -5.4413e-02,
         -1.1121e-02,  4.1767e-02],
        ...,
        [-2.1100e-05,  5.0931e-02,  1.7985e-02,  ..., -2.4838e-02,
          5.2244e-03,  6.2292e-02],
        [ 3.8484e-02,  9.2326e-03, -5.8533e-02,  ...,  4.9661e-02,
          5.3062e-03,  5.2821e-02],
        [-5.2050e-04, -3.6725e-02,  2.8297e-02,  ..., -1.7685e-02,
          5.9900e-02,  4.4516e-02]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.0228, -0.0437, -0.0317, -0.0610,  0.0109, -0.0203,  0.0171, -0.0152,
         0.0326, -0.0188,  0.0552, -0.0379, -0.0010,  0.0485, -0.0387,  0.0120,
         0.0579, -0.0581,  0.0183, -0.0371,  0.0402,  0.0130,  0.0314, -0.0469,
        -0.0056], device='cuda:0', requires_gra

In [10]:
weights_path = '/home/fmr/workspace/scalpel/savedweights/focallosssimplered/model_weights_epoch_4000.pt'

# Initialize model here...

if os.path.exists(weights_path):
    model.load_state_dict(torch.load(weights_path))
    print(f"Loaded model weights from {weights_path}")

Loaded model weights from /home/fmr/workspace/scalpel/savedweights/focallosssimplered/model_weights_epoch_4000.pt


In [2]:
from network import GrayScaleObjectDetector

INPUT_SIZE = (2048, 2048 )
model = GrayScaleObjectDetector(input_size=INPUT_SIZE)

In [5]:
from torchsummary import summary

# Assuming the model is defined and is named 'model'
input_size = (2048, 2048)  # for example, adjust if needed
summary(model, input_size)

RuntimeError: Given groups=1, weight of size [1, 3, 1, 1], expected input[1, 2, 2048, 2048] to have 3 channels, but got 2 channels instead

In [1]:
import torch
from network_fpn import FPNCATSimple


dummy_input = torch.randn(1, 3, 512, 512)
model = FPNCATSimple()
output = model(dummy_input)

Shape of combined tensor: torch.Size([1, 768, 64, 64])
